In [17]:
import sys
sys.path.append("/home/charleshiggins/LazyMDP")
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.atari_wrappers import AtariWrapper
from stable_baselines3 import A2C, PPO
import gymnasium as gym

from lazywrapper import LazyWrapper, LastObservationWrapper
from lazywrapper.custom_callbacks import ActionProportionCallback
# general imports

default_policy = PPO.load("/home/charleshiggins/LazyMDP/baselines/suboptimal_pretrained_policies/ppo_BreakoutNoFrameskip-v4_0")
def make_custom_atari_env(original_env, loaded_default_policy, penalty):
    def _init():
        env = LazyWrapper(LastObservationWrapper(gym.make(original_env)), default_policy=loaded_default_policy, penalty = penalty)
        return env
    return _init
# There already exists an environment generator
# that will make and wrap atari environments correctly.
def make_internal_atari_env(original_env):
    def _init():
        env = gym.make(original_env)
        return env
    return _init
# Here we are also multi-worker training (n_envs=4 => 4 environments)
vec_env_pre = make_atari_env(env_id=make_custom_atari_env("BreakoutNoFrameskip-v4", default_policy, penalty=0.9), n_envs=4, seed=0)
obs = vec_env_pre.reset()
print(obs.shape)
# Frame-stacking with 4 frames
vec_env_post = VecFrameStack(vec_env_pre, n_stack=4)

# model = A2C("CnnPolicy", vec_env, verbose=1)
# model.learn(total_timesteps=5_000)

obs = vec_env_post.reset()

(4, 84, 84, 1)


In [11]:
obs.shape

(4, 84, 84, 4)